In [ ]:
import pandas as pd
import numpy as np
import keyring
import cx_Oracle
import time
import glob
import os
from datetime import date, datetime, timedelta

In [ ]:
time = datetime.now()
weekday = datetime.today().isoweekday()
if weekday == 1 or weekday == 2:
    time = (datetime.now() +timedelta(days=-4))
else:
    time = (datetime.now() +timedelta(days=-2))
year = time.year
month = str(time.month).zfill(2)
day1 = str(time.day).zfill(2)
lbd = (f'{year}{month}{day1}')
fisc_year = lbd[:-2]

In [1]:
# SLEEPER
def download_sleeper(lbd):
    
    SQL_query = f'''
    
    
    '''
    
    
    server = {'GTMIS_db_name': '',
             'GTMIS_host': '',
             'GTMIS_port': '',
             'GTMIS_service_man': ''}
    
    username_GTMIS = keyring.get_password('','')
    dsn = cx_ORacle.makedsn(server['GTMIS_host'], server['GTMIS_port'], service_name=server['GTMIS_service_name'])
    source_conn = cx_Oracle_connect(user=keyring.get_password('Username', 'username'), password=keyring.get_password('GTMIS', username_GTMIS,
                                                                                                                          dsn = dsn, encoding='UTF-8'))
    df_sleeper = pd.read_sql(sql=SQL_query, con=source_conn)
    
    if source_conn is not None:
        source_conn.close()
    return df_sleeper

df_sleeper_raw = download_sleeper(lbd)

df_sleeper_raw.insert(0, 'GCR_GAAP_CODE', '1').zfill(3))
df_sleeper_raw.insert(3, 'GCONS_TP_FLAG', 'N')
df_sleeper_raw.insert(4, 'GRPT_LINE_ID', 'OB4000100')
df_sleeper_raw.insert(9, 'BUSINESS_SCENARIO', 'CMBA')
df_sleeper_raw.insert(10, 'TNR_BUCKET', '1D')
df_sleeper_raw.insert(11, 'TYPE_CODE', '290').zfill(4))
df_sleeper_raw.insert(12, 'SAP_CODE', 'NG0000')
df_sleeper_raw.insert(13, 'CONSO_LVL', '1')
df_sleeper_raw.insert(14, 'LFM_KEY', '630_SLEEPER')

df_sleeper_raw['FISC_YEAR'] = fisc_year
df_sleeper_raw['LFM_KEY'] = df_sleeper_raw['GT_SRC_TYPE_CD'].replace({'Sleeper Collateral': ' DERIVS_MAN_SLEEPER',
                                                                     'Excess Collateral' : 'DERIVS_MAN_EXCESS'})
df_sleeper_raw.rename(columns={'LFM_PNL_AMT_GC': 'LFM_ADJ',
                              'LCR_POST_AMT' : 'LFM_SURPLUS_AMT'}, inplace=True)


# 1 day
df_sleeper_excess_1D = df_sleeper_raw.copy()


# 2 days
df_sleeper_excess_2D = df_sleeper_raw.copy()
df_sleeper_excess_2D['TNR_BUCKER'] = '2D'


# 1 year offset
df_sleeper_excess_1Y = df_sleeper_raw.copy()
df_sleeper_excess_1Y['TNR_BUCKER'] = '13M'
df_sleeper_excess_1Y['LFM_SURPLUS_AMT'] = _sleeper_raw['LFM_SURPLUS_AMT'] * 2 * -1

df_sl_complete = pd.concat([df_sleeper_excess_1D,df_sleeper_excess_2D, df_sleeper_excess_1Y])


In [ ]:
# UCL


def download_ucl(lbd):
    SQL_query = f'''
    '''
    
    server = {'GTMIS_db_name': '',
             'GTMIS_host': '',
             'GTMIS_port': '',
             'GTMIS_service_man': ''}
    
    username_GTMIS = keyring.get_password('','')
    dsn = cx_ORacle.makedsn(server['GTMIS_host'], server['GTMIS_port'], service_name=server['GTMIS_service_name'])
    source_conn = cx_Oracle_connect(user=keyring.get_password('Username', 'username'), password=keyring.get_password('GTMIS', username_GTMIS,
                                                                                                                          dsn = dsn, encoding='UTF-8'))
    df_ucl = pd.read_sql(sql=SQL_query, con=source_conn)
    
    if source_conn is not None:
        source_conn.close()
    return df_ucl

df_ucl_raw = download_ucl(lbd)

df_ucl_raw['GAAP_CODE_CODE'] = df_ucl_raw['GAAP_CODE_CODE'].astype('str').str.zfill(3)
df_ucl_raw['FISC_YEAR'] = fisc_year
df_ucl_raw['CONSOLIDATION_CIRCLE'] = df_ucl_raw['CONSOLIDATION_CIRCLE'].astype('str').str.zfill(2)
df_ucl_raw.insert(21, 'LFM_ADJ', 'Adj_639_UCL')

cf_us_lst = pd.read_csv(r'')
df_icf2 = pd.read_csv(r'')

df_ucl = pd.concat([df_ucl_raw, cf_us_lst,df_icf2])
df_ucl['LFM_CONSO_LVL'] = '610'
df_ucl.drop(columns=['CNS_SCOPE', 'PROD_CD', 'LFM_RPT'], inplace=True)

In [ ]:
df_sleeper_ucl_MW = pd.concat([df_sl_complete, df_ucl])
df_sleeper_ucl_MW.to_excel(r'')